In [ ]:
from utils.dataset import CTMaskDataset
from utils.utils import  plotSomeImages, generateNpySlices, loadMatData, generateSplits
from predict import predict_vol_from_vol
from unet.unet_model import UNet
from random import randint
from eval import jaccard, eval_volumes
import torch
import numpy as np
import scipy as sp

## Create training data.

In [ ]:
# Create list of patient [vol_idxs] from range.
all_idxs = [[a , b] for b in range(1,4) for a in range(1,23)]
# Create list of the class mask_data you want.
mask = ['spine_mask', 'stern_mask', 'pelvi_mask']
# Generate training/data splits from those that have .mat data matching the mask_names. 
val_idxs, trn_idxs = generateSplits(all_idxs, mask_names = mask_names)
# Generate the training .npy files.
generateNpySlices(trn_idxs[0], mask_names = mask_names)

## Verify training data.

In [ ]:
# Create a CTMaskDataset from the training data.
dataset = CTMaskDataset(augment=False)

In [ ]:
# Display a sample from the training data. 
ridx = randint(0, len(dataset))
verify = dataset[ridx]
ct = verify['ct'].squeeze().numpy()
target = verify['target'].squeeze().numpy()
imgs = {'ct': ct, 'target': target}
plotSomeImages(imgs, 1, 2)

## Evaluate model on vol_idxs

In [ ]:
# Set up UNet. Must match model you are loading. 
device = torch.device('cuda')
# n_classes = 1 for binary class, n+1 for multi-class.
net = UNet(n_channels=1, n_classes=4, bilinear=False) 
# Load model from file.
net.load_state_dict(torch.load('C:/.py_workspace/reveal/unet-2D/.runs/multiclass_testing/3class_results/model_state.pth'))
net.to(device)

In [ ]:
# Evaluate from vol_idx list!
vol_idxs = [[2, 1], [2, 3]]
eval_volumes(net, device, vol_idxs)

## Generate prediction volume from model.

In [ ]:
# Set up UNet. Must match model you are loading. 
device = torch.device('cuda')
# n_classes = 1 for binary class, n+1 for multi-class.
net = UNet(n_channels=1, n_classes=4, bilinear=False) 
# Load model from file.
net.load_state_dict(torch.load('C:/.py_workspace/reveal/unet-2D/.runs/multiclass_testing/3class_results/model_state.pth'))
net.to(device)

In [ ]:
# Predict from vol_idx!
vol_idx = [2, 1]
pred_vol = predict_vol_from_vol(net, device, vol_idx)

In [ ]:
# Save as .mat if you want...
# Something like this...
pred_dict = {'bg': pred_vol[0, :, :, :],
            'spine': pred_vol[1, :, :, :],
            'stern': pred_vol[2, :, :, :],
            'pelvi': pred_vol[3, :, :, :]}

sp.io.savemat('mc_pred_p2d1.mat', pred_dict, do_compression=True)